In [1]:
# libraries
import requests
import gzip
from io import BytesIO
import os

import numpy as np
import pandas as pd
from pandas_dq import dq_report

from sklearn.linear_model import LinearRegression

import folium
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from folium.plugins import TimestampedGeoJson

le fichier clean est désormais dispo sur S3 : https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_02.csv

In [2]:
url = "https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_02.csv"
df = pd.read_csv(url)

In [3]:
df.shape

(340387, 26)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340387 entries, 0 to 340386
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   NUM_POSTE                      340387 non-null  int64  
 1   NOM_USUEL                      340387 non-null  object 
 2   LAT                            340387 non-null  float64
 3   LON                            340387 non-null  float64
 4   ALTI                           340387 non-null  int64  
 5   AAAAMM                         340387 non-null  object 
 6   Year                           340387 non-null  int64  
 7   Month                          340387 non-null  int64  
 8   precip_cumul_mensu             327731 non-null  float64
 9   temp_mean_mensu                335882 non-null  float64
 10  temp_max_mensu                 335883 non-null  float64
 11  temp_min_mensu                 335953 non-null  float64
 12  NBJGELEE                      

- Aucune valeur manquante sur notre future target "vent_speed_inst_moy_mensu"
- Pour le moment nous allons travailler sur la position de la station et la vitesse du vent moyenne mensuelle, sans prendre en compte le reste des éléménts

OBJECTIF 1 : déterminer si la force du vent augmente ou pas

- statistiques gobales nationales

In [9]:
# en France : moyenne des vents par mois depuis 1950 
df_monthly = df.groupby("AAAAMM")["vent_speed_inst_moy_mensu"].mean().reset_index()

fig = px.line(df_monthly, x="AAAAMM", y="vent_speed_inst_moy_mensu", title="Vitesse moyenne du vent en France depuis 1950")
fig.show()


Globalement il y a moins de vent en France. Mais il semblerait que la tendance soit à une augmentation depuis 2010. vérifions en affichant les chiffres et sur une période base de 7 ans

In [10]:
# Tableau avec les moyennes mensuelles calculés
df_monthly = df.groupby("AAAAMM")["vent_speed_inst_moy_mensu"].mean().reset_index()
df_monthly

,AAAAMM,vent_speed_inst_moy_mensu
0,1950-01-01,12.921101
1,1950-02-01,21.266667
2,1950-03-01,13.551786
3,1950-04-01,18.109910
4,1950-05-01,12.629204
...,...,...
871,2022-08-01,10.055252
872,2022-09-01,10.688540
873,2022-10-01,10.449250
874,2022-11-01,12.465894


In [11]:
#AAAAMM en datetime
df_monthly["AAAAMM"] = pd.to_datetime(df_monthly["AAAAMM"], format='%Y-%m-%d', errors='coerce')

#périodes d'intérêt
periods = [
    (1953, 1960),
    (1961, 1978),
    (1979, 1985),
    (1986, 1992),
    (1993, 2000),
    (2001, 2008),
    (2009, 2015),
    (2016, 2022)
]

# vitesse moyenne du vent pour chaque période et stockage
mean_wind_speeds = []
for start_year, end_year in periods:
    df_period = df_monthly[(df_monthly["AAAAMM"].dt.year >= start_year) & (df_monthly["AAAAMM"].dt.year <= end_year)]
    mean_wind_speed = df_period['vent_speed_inst_moy_mensu'].mean()
    mean_wind_speeds.append((start_year, end_year, mean_wind_speed))
    print(f"Vitesse moyenne du vent entre {start_year} et {end_year}: {mean_wind_speed}")

# graphique
fig = px.line()

# courbes pour chaque période
for start_year, end_year in periods:
    df_period = df_monthly[(df_monthly["AAAAMM"].dt.year >= start_year) & (df_monthly["AAAAMM"].dt.year <= end_year)]
    fig.add_scatter(x=df_period['AAAAMM'], y=df_period['vent_speed_inst_moy_mensu'], mode='lines', name=f'{start_year}-{end_year}')

# légende
fig.update_layout(title='Vitesse moyenne du vent par période',
                  xaxis_title='Année',
                  yaxis_title='Vitesse moyenne du vent (km/h)')
fig.show()

Vitesse moyenne du vent entre 1953 et 1960: 14.537965109893856
Vitesse moyenne du vent entre 1961 et 1978: 14.864621765790742
Vitesse moyenne du vent entre 1979 et 1985: 14.921481756236986
Vitesse moyenne du vent entre 1986 et 1992: 12.80727255797555
Vitesse moyenne du vent entre 1993 et 2000: 11.496474061836382
Vitesse moyenne du vent entre 2001 et 2008: 11.369316671825331
Vitesse moyenne du vent entre 2009 et 2015: 11.448273793885734
Vitesse moyenne du vent entre 2016 et 2022: 11.654824110893795


Une augmentation du vent semble se dégager en France. Mais une force moyenne de 11.65 km/h est insufisant pour faire tourner une éolienne.
Vérifier la périodicité ou la force serait supérieur 14km/h (force minimale) 

In [13]:
# périodes d'intérêt
periods = [
    (1953, 1960),
    (1961, 1978),
    (1979, 1985),
    (1986, 1992),
    (1993, 2000),
    (2001, 2008),
    (2009, 2015),
    (2016, 2022)
]

counts = {}

# boucle pour chaque période
for start, end in periods:
    period_df = df_monthly[(df_monthly["AAAAMM"].dt.year >= start) & (df_monthly["AAAAMM"].dt.year <= end)]
    count = period_df[period_df["vent_speed_inst_moy_mensu"] > 14].shape[0]
    counts[f"{start}-{end}"] = count

# résultats
print("Nombre de mois où la vitesse moyenne du vent est supérieure à 14 km/h:")
for period, count in counts.items():
    print(f"{period}: {count}")




Nombre de mois où la vitesse moyenne du vent est supérieure à 14 km/h:
1953-1960: 52
1961-1978: 129
1979-1985: 50
1986-1992: 17
1993-2000: 10
2001-2008: 3
2009-2015: 6
2016-2022: 7


Résultat faible, mais au national cela n'est pas très parlant. Examinons les zones de manières régionales

In [16]:
# créer un dataframe avec les données de vent_speed_inst_moy_mensu par date et region
df_date_region = df.groupby(["AAAAMM", "region"])["vent_speed_inst_moy_mensu"].mean().reset_index()


In [20]:
import pandas as pd
import plotly.express as px

# Convert AAAAMM to datetime in the new DataFrame
df_date_region["AAAAMM"] = pd.to_datetime(df_date_region["AAAAMM"], format='%Y-%m-%d', errors='coerce')

# Define periods of interest
periods = [
    (1953, 1960),
    (1961, 1978),
    (1979, 1985),
    (1986, 1992),
    (1993, 2000),
    (2001, 2008),
    (2009, 2015),
    (2016, 2022)
]

# Calculate mean wind speed for each period and region, and store the results
mean_wind_speeds = []
for start_year, end_year in periods:
    for region in df_date_region['region'].unique():
        df_period_region = df_date_region[(df_date_region["AAAAMM"].dt.year >= start_year) & 
                                          (df_date_region["AAAAMM"].dt.year <= end_year) & 
                                          (df_date_region['region'] == region)]
        mean_wind_speed = df_period_region['vent_speed_inst_moy_mensu'].mean()
        mean_wind_speeds.append((start_year, end_year, region, mean_wind_speed))
        print(f"Vitesse moyenne du vent entre {start_year} et {end_year} pour la région {region}: {mean_wind_speed}")

# Create a separate line plot for each region
for region in df_date_region['region'].unique():
    fig = px.line()
    for start_year, end_year in periods:
        df_period_region = df_date_region[(df_date_region["AAAAMM"].dt.year >= start_year) & 
                                          (df_date_region["AAAAMM"].dt.year <= end_year) & 
                                          (df_date_region['region'] == region)]
        fig.add_scatter(x=df_period_region['AAAAMM'], y=df_period_region['vent_speed_inst_moy_mensu'], mode='lines', name=f'{start_year}-{end_year}')
    
    # Update layout for each region
    fig.update_layout(title=f'Vitesse moyenne du vent par période pour la région {region}',
                      xaxis_title='Année',
                      yaxis_title='Vitesse moyenne du vent (km/h)')
    fig.show()

Vitesse moyenne du vent entre 1953 et 1960 pour la région AUVERGNE RHONE ALPES: 12.116666666666665
Vitesse moyenne du vent entre 1953 et 1960 pour la région BOURGOGNE FRANCHE COMTE: 11.245198863636363
Vitesse moyenne du vent entre 1953 et 1960 pour la région BRETAGNE: 19.11282490079365
Vitesse moyenne du vent entre 1953 et 1960 pour la région CENTRE VAL DE LOIRE: 12.669717261904763
Vitesse moyenne du vent entre 1953 et 1960 pour la région GRAND EST: 11.020561868686869
Vitesse moyenne du vent entre 1953 et 1960 pour la région HAUTS DE FRANCE: 15.48863076449014
Vitesse moyenne du vent entre 1953 et 1960 pour la région ILE DE FRANCE: 13.557136656746032
Vitesse moyenne du vent entre 1953 et 1960 pour la région NORMANDIE: 15.971600378787878
Vitesse moyenne du vent entre 1953 et 1960 pour la région NOUVELLE AQUITAINE: 12.687742340125153
Vitesse moyenne du vent entre 1953 et 1960 pour la région OCCITANIE: 15.41145833333333
Vitesse moyenne du vent entre 1953 et 1960 pour la région PAYS DE LA L

In [27]:
# créer un dataframe avec les données de vent_speed_inst_moy_mensu par période et region
df_period_region = pd.DataFrame(mean_wind_speeds, columns=["start_year", "end_year", "region", "mean_wind_speed"])

#visualiser les chiffres trié par région puis par période

df_period_region.sort_values(by=["region", "start_year", "end_year"])


,start_year,end_year,region,mean_wind_speed
0,1953,1960,AUVERGNE RHONE ALPES,12.116667
12,1961,1978,AUVERGNE RHONE ALPES,12.046576
24,1979,1985,AUVERGNE RHONE ALPES,11.020188
36,1986,1992,AUVERGNE RHONE ALPES,9.767065
48,1993,2000,AUVERGNE RHONE ALPES,8.705068
60,2001,2008,AUVERGNE RHONE ALPES,8.927261
72,2009,2015,AUVERGNE RHONE ALPES,9.461614
84,2016,2022,AUVERGNE RHONE ALPES,9.676787
1,1953,1960,BOURGOGNE FRANCHE COMTE,11.245199
13,1961,1978,BOURGOGNE FRANCHE COMTE,11.164878


In [36]:
# Define periods of interest
periods = [
    (1953, 1960),
    (1961, 1978),
    (1979, 1985),
    (1986, 1992),
    (1993, 2000),
    (2001, 2008),
    (2009, 2015),
    (2016, 2022)
]

# Calculate mean wind speed for each period and region
mean_wind_speeds = []
for start, end in periods:
    period_df = df_date_region[(df_date_region["AAAAMM"].dt.year >= start) & 
                               (df_date_region["AAAAMM"].dt.year <= end)]
    period_df_grouped = period_df.groupby("region")["vent_speed_inst_moy_mensu"].mean().reset_index()
    period_df_grouped["period"] = f"{start}-{end}"
    mean_wind_speeds.append(period_df_grouped)

# Concatenate all period DataFrames
df_mean_wind_speeds = pd.concat(mean_wind_speeds)

# Create a grouped bar chart using Plotly Express
fig = px.bar(df_mean_wind_speeds, x="region", y="vent_speed_inst_moy_mensu", color="period",
             title="Vitesse moyenne du vent par région",
             labels={"vent_speed_inst_moy_mensu": "Vitesse moyenne du vent (km/h)", "region": "Région", "period": "Période"},
             barmode="group")

fig.show()

Globalement le vent remonte bien depuis la période "2009". Même depuis la période "1993" sur certaines régions

In [37]:
import pandas as pd
import plotly.express as px

# Convert AAAAMM to datetime in the new DataFrame
df_date_region["AAAAMM"] = pd.to_datetime(df_date_region["AAAAMM"], format='%Y-%m-%d', errors='coerce')

# Define periods of interest
periods = [
    (2001, 2008),
    (2009, 2015),
    (2016, 2022)
]

counts = []

# Loop for each period and region
for start, end in periods:
    for region in df_date_region['region'].unique():
        period_df = df_date_region[(df_date_region["AAAAMM"].dt.year >= start) & 
                                   (df_date_region["AAAAMM"].dt.year <= end) & 
                                   (df_date_region["region"] == region)]
        count = period_df[period_df["vent_speed_inst_moy_mensu"] > 14].shape[0]
        counts.append({"region": region, "period": f"{start}-{end}", "count": count})

# Convert to DataFrame
df_counts = pd.DataFrame(counts)

# Create a bar chart using Plotly Express
fig = px.bar(df_counts, x="region", y="count", color="period",
             title="Nombre de mois où la vitesse moyenne du vent est supérieure à 14 km/h par région",
             labels={"count": "Nombre de mois", "region": "Région", "period": "Période"},
             barmode="group")

fig.show()

Même si la force du vent augmente en moyenne, il peut y moins de mois venteux sur certaines zones, et plus sur d'autres

- intéressons nous aux stations météos :

In [39]:
# calculer le nombre de station différentes
df['NUM_POSTE'].nunique()
print(f"Nombre de stations différentes: {df['NUM_POSTE'].nunique()}")

Nombre de stations différentes: 1129


Est ce que notre dataset est représentatif, et est ce que nos stations météos sont bien réparties sur le territoire ?

In [41]:
# Supprimer les doublons basés sur la colonne 'NUM_POSTE'
df_unique = df.drop_duplicates(subset=['NUM_POSTE'])

# Créer une carte centrée sur la moyenne des coordonnées
carte = folium.Map(location=[df_unique['LAT'].mean(), df_unique['LON'].mean()], zoom_start=6)

# Ajouter un marqueur pour chaque station unique
for idx, row in df_unique.iterrows():
    popup_content = f"Station: {row['NUM_POSTE']}<br>Nom Usuel: {row['NOM_USUEL']}"
    folium.Marker(
        location=[row['LAT'], row['LON']],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"Station: {row['NUM_POSTE']}",
        icon=folium.Icon(icon='wind', prefix='fa', icon_size=(28, 28))
    ).add_to(carte)

# Sauvegarder la carte
carte.save("carte_stations_meteo.html")


In [42]:
# Supprimer les lignes avec la station outlier
df= df[df["NUM_POSTE"] != 17300848]

Définir la station avec le moins de valeur manquante. Elle servira également à la baseline de notre premier modèle

In [43]:
# Filtrer les données pour la période de 1950 à 2022
df_filtered = df[(df['AAAAMM'] >= '1950-01-01') & (df['AAAAMM'] <= '2022-12-31')]

# Compter le nombre de données pour chaque station
station_counts = df_filtered['NUM_POSTE'].value_counts()

# Trouver la station avec le plus de données
most_data_station = station_counts.idxmax()
most_data_count = station_counts.max()

print(f"La station avec le plus de données est {most_data_station} avec {most_data_count} enregistrements.")

La station avec le plus de données est 28070001 avec 876 enregistrements.


In [44]:
# Filtrer les données pour la station 28070001
df_station = df[df["NUM_POSTE"] == 28070001]

# Créer un graphique interactif
fig = px.line(df_station, x="AAAAMM", y="vent_speed_inst_moy_mensu", title="Vitesse moyenne du vent pour la station 28070001")
fig.show()

Sur cette station particulière (Pays-de-la-Loire) nous sommes cohérents avec les stats régionales..visualisons des stations plus exposés aux vents dans le Nord-pas de Calais (62), la Somme(80), le finistère (29)

In [ ]:
# choisir une station ayant au moins 300 enregistrements dans le département 29
stations_29 = df[df["departement_num"] == 29]
stations_29_counts = stations_29["NUM_POSTE"].value_counts()
stations_29_filtered = stations_29_counts[stations_29_counts >= 300]
chosen_station = stations_29_filtered.idxmax()

# Filtrer les données pour la station choisie
df_chosen_station = df[df["NUM_POSTE"] == chosen_station]

# Créer un graphique interactif
fig = px.line(df_chosen_station, x="AAAAMM", y="vent_speed_inst_moy_mensu", title=f"Vitesse moyenne du vent pour la station {chosen_station}")
fig.show()



In [ ]:
# choisir une station ayant au moins 300 enregistrements dans le département80
stations80 = df[df["departement_num"] == 80]
stations80_counts = stations80["NUM_POSTE"].value_counts()
stations80_filtered = stations80_counts[stations80_counts >= 300]
chosen_station = stations80_filtered.idxmax()

# Filtrer les données pour la station choisie
df_chosen_station = df[df["NUM_POSTE"] == chosen_station]

# Créer un graphique interactif
fig = px.line(df_chosen_station, x="AAAAMM", y="vent_speed_inst_moy_mensu", title=f"Vitesse moyenne du vent pour la station {chosen_station}")
fig.show()

In [ ]:
# choisir une station ayant au moins 300 enregistrements dans le département29 = df[df["departement_num"] == 29]
stations_29 = df[df["departement_num"] == 29]
stations29_counts = stations_29["NUM_POSTE"].value_counts()
stations29_filtered = stations29_counts[stations29_counts >= 300]
chosen_station = stations29_filtered.idxmax()

# Filtrer les données pour la station choisie
df_chosen_station = df[df["NUM_POSTE"] == chosen_station]

# Créer un graphique interactif
fig = px.line(df_chosen_station, x="AAAAMM", y="vent_speed_inst_moy_mensu", title=f"Vitesse moyenne du vent pour la station {chosen_station}")
fig.show()

In [ ]:
# choisir une station aléatoire
station = df_unique.sample(22)
station

# Filtrer les données pour la station choisie
df_station = df[df["NUM_POSTE"] == station["NUM_POSTE"].values[0]]

# Créer un graphique interactif
fig = px.line(df_station, x="AAAAMM", y="vent_speed_inst_moy_mensu", title=f"Vitesse moyenne du vent pour la station {station['NUM_POSTE'].values[0]}")
fig.show()

Examinons les mois les plus venteux et les moins venteux

In [ ]:

# List of month names and their corresponding numbers
months = [
    ("janvier", 1), ("février", 2), ("mars", 3), ("avril", 4),
    ("mai", 5), ("juin", 6), ("juillet", 7), ("août", 8),
    ("septembre", 9), ("octobre", 10), ("novembre", 11), ("décembre", 12)
]

# Loop through the months and create the plots
for month_name, month_num in months:
    df_month = df_monthly[df_monthly["AAAAMM"].dt.month == month_num]
    fig = px.line(df_month, x="AAAAMM", y="vent_speed_inst_moy_mensu", title=f"Vitesse moyenne du vent en France pour le mois de {month_name}")
    fig.show()

In [45]:
#classer les résultat du mois le plus venteux au moins venteux
df_monthly["month"] = df_monthly["AAAAMM"].dt.month
df_monthly["month_name"] = df_monthly["AAAAMM"].dt.strftime("%B")
df_monthly_grouped = df_monthly.groupby("month_name")["vent_speed_inst_moy_mensu"].mean().reset_index()
df_monthly_grouped = df_monthly_grouped.sort_values(by="vent_speed_inst_moy_mensu", ascending=False)

# Create a bar chart using Plotly Express
fig = px.bar(df_monthly_grouped, x="month_name", y="vent_speed_inst_moy_mensu",
             title="Vitesse moyenne du vent en France par mois",
             labels={"vent_speed_inst_moy_mensu": "Vitesse moyenne du vent (km/h)", "month_name": "Mois"},
             color="month_name")

fig.show()

In [47]:
# Extract month and month name
df_date_region["month"] = df_date_region["AAAAMM"].dt.month
df_date_region["month_name"] = df_date_region["AAAAMM"].dt.strftime("%B")

# Group by region and month name, then calculate the mean wind speed
df_date_region_grouped = df_date_region.groupby(["region", "month_name"])["vent_speed_inst_moy_mensu"].mean().reset_index()

# Sort the results by region and then by mean wind speed in descending order
df_date_region_grouped = df_date_region_grouped.sort_values(by=["region", "vent_speed_inst_moy_mensu"], ascending=[True, False])

# Create a bar chart using Plotly Express
fig = px.bar(df_date_region_grouped, x="region", y="vent_speed_inst_moy_mensu",
             title="Vitesse moyenne du vent par région et par mois",
             labels={"vent_speed_inst_moy_mensu": "Vitesse moyenne du vent (km/h)", "region": "Région", "month_name": "Mois"},
             color="month_name",
             category_orders={"month_name": ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]})

# Update layout for better readability
fig.update_layout(xaxis={'categoryorder':'total descending'},
                  yaxis_title="Vitesse moyenne du vent (km/h)",
                  xaxis_title="Région",
                  legend_title="Mois")

fig.show()

- nos stations sont bien réparties sur le territoire français et nous eprmettent une analyse globale assez complète
- en France le vent augmente globalement depuis 2000/2010.
- est-ce dû au réchauffement climatique ou à d'autres phénomènes ? la question reste ouverte
- mais le vent moyen n'est pas forcément très élevé et ne suffirait pas à faire tourner une éolinenne
- certaines régions sont clairement identifiés comme à fort potentiel avec des vents plus forts que la moyenne nationale
- que même si le vent n'augmente pas dans certaines régions il reste en moyenne suffisament élevé pour faire tourner une éolienne (ex :; Pays-de-la-Loire)
- certaines régions seraient intéressante à analyser pour l'implémentation car un vent moyen en augmentation malgrès une moyenne anyuelle assez loin des 14km/h
- il reste des zones peu propices à l'implémentation d'éolienne en France même si le vent augmentait
- les mois les plus venteux en France sont janvier, février et mars
- les mois le moins venteux en France sont juillet, août et septembre

- les fortes variations locales de la vitesse du vent peuvent être un frein c'est pourquoi nous allons analyser les données de vent station météo par station météo
- afin de créer un modèle assez précis pour tenter determiner la force du vent à 7 ans
- nous supperposerons ensuite les données prédites sur une carte comportant les éoliennes déjà existantes, et un calque des zones "autorisés" à l'implémentation